# Demo: Bayesian Linear Regression on the Diabetes Dataset

This notebook walks through Bayesian Linear Regression with predictive uncertainty. We use scikit-learn's `BayesianRidge` on the diabetes dataset and visualize the posterior predictive mean and variance.

## Why Bayesian instead of deterministic regression?
- **Deterministic linear regression** finds a single best-weight vector.
- **Bayesian linear regression** treats weights as distributions and returns a *posterior* given the data.
- Predictions become probability distributions, giving both a mean and uncertainty (standard deviation).
- Uncertainty is especially helpful in low-data regimes: wider intervals remind us to be cautious.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sns.set(style="whitegrid")

## Load and inspect the data

In [ ]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target
print(diabetes.DESCR.split('\n')[0])
print(f"Features shape: {X.shape}, Target shape: {y.shape}")

## Train/test split and scaling
Bayesian models also benefit from standardized features because priors are easier to interpret on comparable scales.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Fit Bayesian Ridge Regression
`BayesianRidge` assumes Gaussian priors on weights and the noise term. After observing data, it returns the posterior distribution of the weights.

In [ ]:
model = BayesianRidge()
model.fit(X_train_scaled, y_train)

# Predictive mean and standard deviation
mean_pred, std_pred = model.predict(X_test_scaled, return_std=True)

## Evaluation metrics

In [ ]:
mse = mean_squared_error(y_test, mean_pred)
mae = mean_absolute_error(y_test, mean_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, mean_pred)
print({"MSE": mse, "MAE": mae, "RMSE": rmse, "R2": r2})

## Visualize actual vs predicted

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, mean_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

## Predictive uncertainty band
We plot the predictive mean along with ±2 standard deviations (approximate 95% credible interval under Gaussian assumptions).

In [ ]:
plt.figure(figsize=(10, 5))
indices = np.arange(len(mean_pred))
plt.plot(indices, mean_pred, label='Predictive mean')
plt.fill_between(indices, mean_pred - 2*std_pred, mean_pred + 2*std_pred,
                 color='orange', alpha=0.3, label='~95% credible interval')
plt.scatter(indices, y_test, color='steelblue', s=20, alpha=0.6, label='Actual')
plt.xlabel('Test sample index')
plt.ylabel('Disease progression')
plt.legend()
plt.title('Predictive Uncertainty')
plt.show()

## Interpreting uncertainty
- Narrow intervals suggest the model is confident (data support the prediction).
- Wide intervals suggest caution; collect more data or revisit feature engineering.
- You can experiment with `alpha_1`, `lambda_1`, or swap in `ARDRegression` to see how priors change the spread.